# Can I predict how much people will spend online?


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv('country_city_sessionduration.csv',
                skiprows=5,
                #skipfooter=1,
                engine='python')
print(df.shape)
#df.head(5)

(3566, 6)


In [2]:
##Turn prices into numbers not strings
print(type(df['Revenue'][0]))
df['Revenue'] = df['Revenue'].str.replace(',','')
df['Revenue'] = df['Revenue'].str.strip('[\$,]')

df['Revenue'] = pd.to_numeric(df['Revenue'])
print(type(df['Revenue'][0]))

#display(df.head(5))

<class 'str'>
<class 'numpy.float64'>


# Baselines


In [3]:
### My baselines:
avg_rev = df['Revenue'].mean().round(2)
print(f'My average revenue is: ${avg_rev}')
### I want to predict what factors affect revenue

My average revenue is: $958.38


## Dropping non-unique Transaction IDs

In [4]:
df.shape

(3566, 6)

In [5]:
df['Transaction ID'].value_counts().head()
### I have some non unique transaction ID's

#9306    11
#7878     4
#7849     4
#8472     4
#9648     3
Name: Transaction ID, dtype: int64

In [6]:
## I think these are some kind of refund/second transaction becuase the revenues are all the same.
## Accordingly, I'm dropping all except the "last" aka first time it appears.
## I'm also doing this so I can merge on unique IDs below.
df.drop_duplicates(subset ="Transaction ID", keep = 'last', inplace = True)
df.shape

(3532, 6)

In [7]:
df['Transaction ID'].value_counts()

#10253    1
#12063    1
#11252    1
#11869    1
#10185    1
         ..
#10098    1
#11174    1
#12055    1
#9041     1
#8619     1
Name: Transaction ID, Length: 3532, dtype: int64

# Add user type, region, metro

In [8]:
df2 = pd.read_csv('usertype_region_metro.csv',
                skiprows=5,
                skipfooter=2,
                engine='python')

print(df2.shape)
#df2

(3555, 6)


In [9]:
df2.drop_duplicates(subset ="Transaction ID", keep = 'last', inplace = True)
df2 = df2.drop(columns=['Date'])
df2 = df2.drop(columns=['Revenue'])
print(df2.shape)
#df2

(3531, 4)


In [10]:
df2['Transaction ID'].value_counts()

#10253    1
#12063    1
#11252    1
#11869    1
#10185    1
         ..
#10098    1
#11174    1
#12055    1
#9041     1
#8619     1
Name: Transaction ID, Length: 3531, dtype: int64

In [11]:
df = df.merge(df2, how='outer', on='Transaction ID', validate='one_to_one')
print(df.shape)
#df

(3532, 9)


# Add Day of Week, Days to Transaction, Days Since Last Session

In [12]:
df3 = pd.read_csv('days.csv',
                skiprows=5,
                skipfooter=2,
                engine='python')

print(df3.shape)
#df3

(3560, 6)


In [13]:
df3.drop_duplicates(subset ="Transaction ID", keep = 'last', inplace = True)
df3 = df3.drop(columns=['Date'])
df3 = df3.drop(columns=['Revenue'])
print(df3.shape)
print(df3['Transaction ID'].value_counts())
#df3

(3531, 4)
#10253    1
#12063    1
#11252    1
#11869    1
#10185    1
         ..
#10098    1
#11174    1
#12055    1
#9041     1
#8619     1
Name: Transaction ID, Length: 3531, dtype: int64


In [14]:
df = df.merge(df3, how='outer', on='Transaction ID', validate='one_to_one')
print(df.shape)
#df

(3532, 12)


# Add Device, Number of Sessions, Page Depth


In [15]:
df4 = pd.read_csv('device_sessions_pagedepth.csv',
                skiprows=5,
                skipfooter=2,
                engine='python')

print(df4.shape)
#df4

(3565, 6)


In [16]:
df4.drop_duplicates(subset ="Transaction ID", keep = 'last', inplace = True)
df4 = df4.drop(columns=['Date'])
df4 = df4.drop(columns=['Revenue'])
print(df4.shape)
print(df4['Transaction ID'].value_counts())
#df4

(3531, 4)
#10253    1
#12063    1
#11252    1
#11869    1
#10185    1
         ..
#10098    1
#11174    1
#12055    1
#9041     1
#8619     1
Name: Transaction ID, Length: 3531, dtype: int64


In [17]:
df = df.merge(df4, how='outer', on='Transaction ID', validate='one_to_one')
print(df.shape)
df

(3532, 15)


,Transaction ID,Date,Country,City,Session Duration,Revenue,User Type,Region,Metro,Day of Week,Days to Transaction,Days Since Last Session,Device Category,Count of Sessions,Page Depth
0,#12144,20200301,United States,Waterville Valley,593 seconds,982.86,New Visitor,New Hampshire,Burlington VT-Plattsburgh NY,0.0,0.0,0.0,desktop,1.0,15.0
1,#12145,20200301,United States,Dallas,336 seconds,2671.15,Returning Visitor,Texas,Dallas-Ft. Worth TX,0.0,0.0,0.0,tablet,4.0,8.0
2,#12146,20200301,United States,Seattle,267 seconds,990.82,New Visitor,Washington,Seattle-Tacoma WA,0.0,0.0,0.0,mobile,1.0,8.0
3,#12147,20200301,United States,Orlando,0 seconds,123.01,Returning Visitor,Florida,Orlando-Daytona Beach-Melbourne FL,0.0,0.0,0.0,mobile,3.0,1.0
4,#12148,20200301,United States,Champaign,1359 seconds,105.10,Returning Visitor,Illinois,Champaign & Springfield-Decatur IL,0.0,0.0,0.0,desktop,3.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3527,#7851,20180105,United Kingdom,London,589 seconds,296.35,New Visitor,England,London,5.0,0.0,0.0,desktop,1.0,11.0
3528,#7852,20180105,United States,Madison,402 seconds,80.11,New Visitor,Wisconsin,Madison WI,5.0,0.0,0.0,desktop,1.0,20.0
3529,#7848,20180103,United States,Manteno,104 seconds,2653.14,Returning Visitor,Illinois,Chicago IL,3.0,0.0,0.0,desktop,7.0,5.0
3530,#7849,20180103,United States,Willow Grove,1205 seconds,119.54,Returning Visitor,Pennsylvania,Philadelphia PA,4.0,6.0,0.0,desktop,6.0,1.0


In [18]:
###Output to csv
df.to_csv('mergedGAdata.csv')